<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Keras_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import tensorflow.keras as keras
import sklearn as sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import MaxPooling1D
from keras.layers import Conv1D
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras import layers
from google.colab import files
!pip install talos
import talos
import datetime as dt

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
# CARGAMOS LOS CSV
url_train = 'https://raw.githubusercontent.com/JulioCastillo1/TP1-Datos/master/train.csv'
url_test = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test.csv'

train_df = pd.read_csv(url_train)
test_df = pd.read_csv(url_test)
train_df = train_df.drop(['keyword', 'location','id'], axis = 1)
train_df.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
train_df.shape

(7613, 2)

In [4]:
x_train, x_test, y_train, y_test = train_test_split(train_df['text'],train_df['target'],random_state=7,test_size = 0.20)
x_train = train_df['text']
y_train = train_df['target']

In [5]:
#vectorizer = CountVectorizer()
#vectorizer.fit(x_train)

#x_train = vectorizer.transform(x_train)
#x_test = vectorizer.transform(x_test)
#x_train

In [6]:
#input_dim = x_train.shape[1]

#model = Sequential()
#model.add(layers.Dense(4,input_dim=input_dim,activation='relu'))
#model.add(layers.Dense(1,activation = 'sigmoid'))

In [7]:
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.summary()

In [8]:
#history = model.fit(x_train,y_train,epochs = 10,verbose = 1,validation_data = (x_test,y_test),batch_size=15)

In [9]:
def modelardo(x_train,y_train,x_test,y_test,params):
  vectorizer = CountVectorizer()
  vectorizer.fit(x_train)

  x_train = vectorizer.transform(x_train)
  x_test = vectorizer.transform(x_test)
  input_dim = x_train.shape[1]
  model = Sequential()
  model.add(layers.Dense(params['first neuron units'],input_dim=input_dim,activation=params['first neuron activation']))
  model.add(layers.Dense(1,activation = 'sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  out = model.fit(x_train,y_train,epochs = params['epochs'],verbose = 0,validation_data = (x_test,y_test),batch_size=params['batch size'])
  return out,model

In [10]:
p = {
    'first neuron units': (2,20,1),
    'first neuron activation': ['relu','elu','sigmoid','softmax','tanh','exponential'],
    'epochs': [1,2,3,4,5,6,7,10,20,100],
    'batch size': (5,100,5)
}

In [11]:
#loss, accuracy = model.evaluate(x_train, y_train, verbose=False)
#print("Training Accuracy: {:.4f}".format(accuracy))
#loss, accuracy = model.evaluate(x_test, y_test, verbose=False)
#print("Testing Accuracy:  {:.4f}".format(accuracy))

In [12]:
x_train.shape
t = talos.Scan(x_train,y_train,params=p,model=modelardo,experiment_name='diabetes',time_limit=(dt.datetime.now() + dt.timedelta(minutes = 10)).strftime("%Y-%m-%d %H:%M"),random_method = 'quantum')
#Este proceso tarda 1,5hs aprox en colab. Y tiene muy poquitos hiperparámetros para tunear

  0%|          | 0/300 [00:00<?, ?it/s]

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



 14%|█▍        | 43/300 [10:32<1:03:02, 14.72s/it]


In [13]:
print('test')

test


In [14]:
experiment_data = t.data
experiment_data.head()

,round_epochs,val_loss,val_accuracy,loss,accuracy,batch size,epochs,first neuron activation,first neuron units
0,1,0.484440,0.804729,0.584518,0.740664,5,1,relu,2
1,1,0.471945,0.801664,0.574076,0.744793,5,1,elu,2
2,1,0.586633,0.773205,0.638376,0.712141,5,1,sigmoid,2
3,1,0.531476,0.792032,0.602357,0.732220,5,1,softmax,2
4,1,0.471479,0.815674,0.569535,0.744417,5,1,tanh,2


In [15]:
#experiment_data['val_accuracy'].idxmax()

In [16]:
#experiment_data.iloc[205]

In [17]:
vectorizer = CountVectorizer()
vectorizer.fit(train_df['text'])

data_train = vectorizer.fit_transform(train_df['text']).toarray()


In [18]:
# Divide el set en train y validation, y entrena con el modelo tomando los hiperparámetros sugeridos

x_train, x_validation, y_train, y_validation = train_test_split(data_train,train_df['target'],random_state=7,test_size = 0.20)


input_dim = x_train.shape[1]

best_model = Sequential()
best_model.add(layers.Dense(2,input_dim=input_dim,activation='elu'))
best_model.add(layers.Dense(1,activation = 'sigmoid'))
best_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
out = best_model.fit(x_train,y_train,epochs = 5,verbose = 0,validation_data = (x_validation,y_validation),batch_size=50)

loss, accuracy = best_model.evaluate(x_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = best_model.evaluate(x_validation, y_validation, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9144
Testing Accuracy:  0.8188


Miren lo buenos que son los resultados con UNA SOLA CAPA DENSE.
Sin embedding nada.
Imaginense los resultados con capas convolucionales y un buen embedding. Eso es lo que les invito a añadir a uds que estuvieron practicando con eso. Yo tuve muchos problemas para hacer un submit, porque no entiendo bien como funciona el CountVectorizer. Voy a seguir con mecanismos de gradient boosting ahora. Por favor hagamos tuning de hiperparámetros a todos los modelos que usemos para poder exprimirlos bien. Sino tenemos una version muy light de los modelos.

In [19]:
#Reentrena con el set completo y realiza predicción.

out = best_model.fit(data_train,train_df['target'],epochs = 5,verbose = 1,batch_size=62)
data_test = vectorizer.transform(test_df['text']).toarray()
prediction=best_model.predict_classes(data_test, batch_size=512)
prediction

Epoch 1/5
7613/7613 [==============================] - 2s 249us/step - loss: 0.3228 - accuracy: 0.8945
Epoch 2/5
7613/7613 [==============================] - 2s 245us/step - loss: 0.2809 - accuracy: 0.9090
Epoch 3/5
7613/7613 [==============================] - 2s 241us/step - loss: 0.2469 - accuracy: 0.9242
Epoch 4/5
7613/7613 [==============================] - 2s 242us/step - loss: 0.2183 - accuracy: 0.9368
Epoch 5/5
7613/7613 [==============================] - 2s 243us/step - loss: 0.1933 - accuracy: 0.9465


array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

In [20]:
# Genera archivo submit.

test_df['target'] = prediction
export = pd.DataFrame()
export['id'] = test_df['id']
export['target'] = test_df['target']
export = export.set_index('id')
export.to_csv('submit.csv')
#files.download('submit.csv')